# Predicting Arrival Delay of flights

We are predicting the arrival delay of flights in minutes for November and December 2019, using Random Forest Regeressor. 

In [133]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('/Users/sayalishelke/Downloads/December.csv')
df2= pd.read_csv('/Users/sayalishelke/Downloads/November.csv')
df= pd.concat([df1,df2])

In [134]:
df.head()

,YEAR,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,Unnamed: 13
0,2019,2019-12-08,WN,15016,"St. Louis, MO",14679,"San Diego, CA",1110,1107.0,0.0,1315,1333.0,18.0,NaN
1,2019,2019-12-08,WN,15016,"St. Louis, MO",14683,"San Antonio, TX",1210,1218.0,8.0,1435,1423.0,-12.0,NaN
2,2019,2019-12-08,WN,15016,"St. Louis, MO",14683,"San Antonio, TX",2145,2140.0,0.0,5,2351.0,-14.0,NaN
3,2019,2019-12-08,WN,15016,"St. Louis, MO",14747,"Seattle, WA",930,925.0,0.0,1205,1141.0,-24.0,NaN
4,2019,2019-12-08,WN,15016,"St. Louis, MO",14771,"San Francisco, CA",1805,1822.0,17.0,2035,2038.0,3.0,NaN


In [135]:
df.drop(labels=['Unnamed: 13'],inplace=True,axis=1)

In [136]:
df = df[df['OP_UNIQUE_CARRIER']=='NK'] #NK code stands for Spirit Airways

In [137]:
df.reset_index(drop=True, inplace=True) 

In [138]:
df.head()

,YEAR,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_TIME,ARR_DELAY
0,2019,2019-12-01,NK,11697,"Fort Lauderdale, FL",13930,"Chicago, IL",1130,1148.0,18.0,1348,1413.0,25.0
1,2019,2019-12-02,NK,11697,"Fort Lauderdale, FL",13930,"Chicago, IL",1130,1124.0,0.0,1348,1335.0,-13.0
2,2019,2019-12-05,NK,11697,"Fort Lauderdale, FL",13930,"Chicago, IL",1139,1136.0,0.0,1400,1351.0,-9.0
3,2019,2019-12-06,NK,11697,"Fort Lauderdale, FL",13930,"Chicago, IL",1139,1134.0,0.0,1400,1357.0,-3.0
4,2019,2019-12-07,NK,11697,"Fort Lauderdale, FL",13930,"Chicago, IL",1139,1137.0,0.0,1400,1346.0,-14.0


In [139]:
s3 = df.drop(['ARR_DELAY'], axis=1)

In [140]:
categorical_feature_mask = s3.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = s3.columns[categorical_feature_mask].tolist()

In [141]:
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()

In [142]:
s3[categorical_cols] = s3[categorical_cols].apply(lambda col: le.fit_transform(col))

In [143]:
y = df['ARR_DELAY']
X = s3

In [144]:
X =X.fillna(0)
y = y.fillna(0)

In [145]:
X.shape

(34737, 12)

In [146]:
y.shape

(34737,)

In [147]:
#split dataset into training and testing set
from sklearn import metrics
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [151]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [152]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 4.920581462291308
Mean Squared Error: 77.93959228554981
Root Mean Squared Error: 8.82834029053875


In [153]:
#R-squared value
print(regressor.score(X_test,y_test))

0.954381157289665


In [159]:
y_pred[0:10] #first 10 values

array([ 33.22,  -4.85, -23.86,   5.77, -11.17,  -3.71, -13.11,  -8.49,
       -10.85,  -1.06])

The negative values above indicate early flights. 

We can see that the R-squared value is 0.95 which is very good. RMSE of 8.82 indicates that the regression is giving quite good results. 